In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from fastai.vision import *
from fastai.metrics import accuracy
from fastai.basic_data import *
from skimage.util import montage
import pandas as pd
from torch import optim
import re
import numpy as np
import torch

In [ ]:
# https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py
def apk(actual, predicted, k=10):
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

def map5(preds, targs):
    predicted_idxs = preds.sort(descending=True)[1]
    top_5 = predicted_idxs[:, :5]
    res = mapk([[t] for t in targs.cpu().numpy()], top_5.cpu().numpy(), 5)
    return torch.tensor(res)

def top_5_preds(preds): return np.argsort(preds.numpy())[:, ::-1][:, :5]

def top_5_pred_labels(preds, classes):
    top_5 = top_5_preds(preds)
    labels = []
    for i in range(top_5.shape[0]):
        labels.append(' '.join([classes[idx] for idx in top_5[i]]))
    return labels

def create_submission(preds, data, name, classes=None):
    if not classes: classes = data.classes
    sub = pd.DataFrame({'Image': [path.name for path in data.test_ds.x.items]})
    sub['Id'] = top_5_pred_labels(preds, classes)
    sub.to_csv(f'{name}.csv.gz', index=False, compression='gzip')

In [ ]:
import fastai
from fastprogress import force_console_behavior
import fastprogress
fastprogress.fastprogress.NO_BAR = True
master_bar, progress_bar = force_console_behavior()
fastai.basic_train.master_bar, fastai.basic_train.progress_bar = master_bar, progress_bar

In [ ]:
df = pd.read_csv('../input/humpback-whale-identification/train.csv')
val_fns = {'69823499d.jpg'}

In [ ]:
fn2label = {row[1].Image: row[1].Id for row in df.iterrows()}
path2fn = lambda path: re.search('\w*\.jpg$', path).group(0)

In [ ]:
name = f'res50-full-train'

In [ ]:
SZ = 224
BS = 64
NUM_WORKERS = 0
SEED=0

In [ ]:
data = (
    ImageItemList
        .from_df(df[df.Id != 'new_whale'], '../input/humpback-whale-identification/train', cols=['Image'])
        .split_by_valid_func(lambda path: path2fn(path) in val_fns)
        .label_from_func(lambda path: fn2label[path2fn(path)])
        .add_test(ImageItemList.from_folder('../input/humpback-whale-identification/test'))
        .transform(get_transforms(do_flip=False), size=SZ, resize_method=ResizeMethod.SQUISH)
        .databunch(bs=BS, num_workers=NUM_WORKERS, path='.')
        .normalize(imagenet_stats)
)

In [ ]:
%%time

learn = create_cnn(data, models.resnet50, lin_ftrs=[2048])
learn.clip_grad();

# learn.fit_one_cycle(7, 1e-2)
learn.save(f'{name}-stage-1')

# learn.unfreeze()

max_lr = 1e-3
lrs = [max_lr/100, max_lr/10, max_lr]

# learn.fit_one_cycle(10, lrs)
learn.save(f'{name}-stage-2')

In [ ]:
SZ = 224 * 2
BS = 64 // 4
NUM_WORKERS = 0
SEED=0

In [ ]:
data = (
    ImageItemList
        .from_df(df[df.Id != 'new_whale'], '../input/humpback-whale-identification/train', cols=['Image'])
        .split_by_valid_func(lambda path: path2fn(path) in val_fns)
        .label_from_func(lambda path: fn2label[path2fn(path)])
        .add_test(ImageItemList.from_folder('../input/humpback-whale-identification/test'))
        .transform(get_transforms(do_flip=False), size=SZ, resize_method=ResizeMethod.SQUISH)
        .databunch(bs=BS, num_workers=NUM_WORKERS, path='.')
        .normalize(imagenet_stats)
)

In [ ]:
%%time
learn = create_cnn(data, models.resnet50, lin_ftrs=[2048])
learn.clip_grad();
# learn.load(f'{name}-stage-2')
learn.freeze_to(-1)

# learn.fit_one_cycle(4, 1e-2 / 4)
# learn.save(f'{name}-stage-3')

learn.unfreeze()

max_lr = 1e-3 / 4
lrs = [max_lr/100, max_lr/10, max_lr]

# learn.fit_one_cycle(6, lrs)
# learn.save(f'{name}-stage-4')

In [ ]:
!cp ../input/models2/* models/

In [ ]:
# with oversampling
df = pd.read_csv('../input/sample/oversampled_train_and_val.csv')

In [ ]:
data = (
    ImageItemList
        .from_df(df, '../input/humpback-whale-identification/train', cols=['Image'])
        .split_by_valid_func(lambda path: path2fn(path) in val_fns)
        .label_from_func(lambda path: fn2label[path2fn(path)])
        .add_test(ImageItemList.from_folder('../input/humpback-whale-identification/test'))
        .transform(get_transforms(do_flip=False), size=SZ, resize_method=ResizeMethod.SQUISH)
        .databunch(bs=BS, num_workers=NUM_WORKERS, path='.')
        .normalize(imagenet_stats)
)

In [ ]:
%%time
learn = create_cnn(data, models.resnet50, lin_ftrs=[2048])
learn.clip_grad();
learn.load(f'{name}-stage-4')
learn.freeze_to(-1)

learn.fit_one_cycle(2, 1e-2 / 4)
learn.save(f'{name}-stage-5')

learn.unfreeze()

max_lr = 1e-3 / 4
lrs = [max_lr/100, max_lr/10, max_lr]

learn.fit_one_cycle(4, lrs)
learn.save(f'{name}-stage-6')

## Predict

In [ ]:
preds, _ = learn.get_preds(DatasetType.Test)

In [ ]:
preds = torch.cat((preds, torch.ones_like(preds[:, :1])), 1)

In [ ]:
preds[:, 5004] = 0.06

In [ ]:
classes = learn.data.classes + ['new_whale']

In [ ]:
create_submission(preds, learn.data, name, classes)

In [ ]:
pd.read_csv(f'{name}.csv.gz').head()

In [ ]:
pd.read_csv(f'{name}.csv.gz').Id.str.split().apply(lambda x: x[0] == 'new_whale').mean()

In [ ]:
!kaggle competitions submit -c humpback-whale-identification -f subs/{name}.csv.gz -m "{name}"

In [ ]:
!cat res50-full-train.csv.gz